In [23]:
import pandas as pd
import os
import re
file_path="C:\\Users\\bedlamini\\Documents\\VAT Schedules\\RES Files\\NEW RES -ERS Sales Purchase VAT MSCO.xlsx"
df=pd.read_excel(file_path,sheet_name=["INPUT TAX","OUTPUT_TAX"], header=0)

input_df=df.get("INPUT TAX")
output_df=df.get("OUTPUT_TAX")
input_df['index_column']=input_df.index

In [24]:
input_df.head(2)


,CompanyCode,CompanyCodeName,Document Date,VAT Registration Number,Supplier Name,Document Number,Reference,Tax Code,Tax Code Description,Tax Amount,Tax BaseAmount,index_column
0,2000,Mhlume Sugar Company Ltd,20220805.0,100108176,Swazi-Trac Pty Ltd,1.900286e+09,136763,V7,15% Input VAT,10177.94,67852.90,0
1,2000,Mhlume Sugar Company Ltd,20221019.0,100108176,Swazi-Trac Pty Ltd,5.100175e+09,139441,V7,15% Input VAT,678.96,4526.42,1


In [25]:
input_df_pv=input_df.pivot_table(index=['index_column','Document Date','VAT Registration Number','Supplier Name','Reference','Tax BaseAmount'], columns='Tax Code Description',values=['Tax Amount']).stack(level=0).reset_index()

input_df_pv.head(10)


Tax Code Description,index_column,Document Date,VAT Registration Number,Supplier Name,Reference,Tax BaseAmount,level_6,0% Input Tax,0% Non Registered Vendors (SWD),15% Input VAT,Exempt Input Tax
0,0,20220805.0,100108176,Swazi-Trac Pty Ltd,136763,67852.90,Tax Amount,NaN,NaN,10177.94,NaN
1,1,20221019.0,100108176,Swazi-Trac Pty Ltd,139441,4526.42,Tax Amount,NaN,NaN,678.96,NaN
2,2,20230203.0,100240818,Serec Radio Ltd,25025,2349.40,Tax Amount,NaN,NaN,352.41,NaN
3,3,20230301.0,100232150,Augusta Enterprises Swaziland,IN128486,60904.84,Tax Amount,NaN,NaN,9135.73,NaN
4,4,20230324.0,100232150,Augusta Enterprises Swaziland,IN128678,141309.00,Tax Amount,NaN,NaN,21196.35,NaN
5,5,20230417.0,100232150,Augusta Enterprises Swaziland,IN131458,301758.48,Tax Amount,NaN,NaN,45263.77,NaN
6,6,20230420.0,100199376,DHL International-Swaziland,MTSIR00026083,12558.82,Tax Amount,0.0,NaN,NaN,NaN
7,7,20230420.0,100199376,DHL International-Swaziland,D0007562,-12558.82,Tax Amount,0.0,NaN,NaN,NaN
8,8,20230510.0,100182626,Quality Investments t/a Quality Eng,7212,7500.00,Tax Amount,NaN,NaN,1125.00,NaN
9,9,20230510.0,100182626,Quality Investments t/a Quality Eng,7208,5500.00,Tax Amount,NaN,NaN,825.00,NaN


In [26]:
input_df_pv=input_df_pv.assign(Standardratedimports='',Adjustment='',Adjustmentother='',DescriptionGood='')
input_df_pv = input_df_pv.fillna(value=0)
input_df_pv['VAT']=input_df_pv['0% Input Tax'] + input_df_pv['15% Input VAT']
input_df_pv.head(5)

Tax Code Description,index_column,Document Date,VAT Registration Number,Supplier Name,Reference,Tax BaseAmount,level_6,0% Input Tax,0% Non Registered Vendors (SWD),15% Input VAT,Exempt Input Tax,Standardratedimports,Adjustment,Adjustmentother,DescriptionGood,VAT
0,0,20220805.0,100108176,Swazi-Trac Pty Ltd,136763,67852.90,Tax Amount,0.0,0.0,10177.94,0.0,,,,,10177.94
1,1,20221019.0,100108176,Swazi-Trac Pty Ltd,139441,4526.42,Tax Amount,0.0,0.0,678.96,0.0,,,,,678.96
2,2,20230203.0,100240818,Serec Radio Ltd,25025,2349.40,Tax Amount,0.0,0.0,352.41,0.0,,,,,352.41
3,3,20230301.0,100232150,Augusta Enterprises Swaziland,IN128486,60904.84,Tax Amount,0.0,0.0,9135.73,0.0,,,,,9135.73
4,4,20230324.0,100232150,Augusta Enterprises Swaziland,IN128678,141309.00,Tax Amount,0.0,0.0,21196.35,0.0,,,,,21196.35


In [27]:
column_order=['Document Date','VAT Registration Number','Supplier Name','Reference','DescriptionGood','Exempt Input Tax','0% Input Tax','0% Non Registered Vendors (SWD)','15% Input VAT','Standardratedimports','Adjustment','Adjustmentother','VAT','Tax BaseAmount']
df=input_df_pv.reindex(column_order,axis=1)
df.head(2)

Tax Code Description,Document Date,VAT Registration Number,Supplier Name,Reference,DescriptionGood,Exempt Input Tax,0% Input Tax,0% Non Registered Vendors (SWD),15% Input VAT,Standardratedimports,Adjustment,Adjustmentother,VAT,Tax BaseAmount
0,20220805.0,100108176,Swazi-Trac Pty Ltd,136763,,0.0,0.0,0.0,10177.94,,,,10177.94,67852.90
1,20221019.0,100108176,Swazi-Trac Pty Ltd,139441,,0.0,0.0,0.0,678.96,,,,678.96,4526.42


In [28]:
df.head(10)

Tax Code Description,Document Date,VAT Registration Number,Supplier Name,Reference,DescriptionGood,Exempt Input Tax,0% Input Tax,0% Non Registered Vendors (SWD),15% Input VAT,Standardratedimports,Adjustment,Adjustmentother,VAT,Tax BaseAmount
0,20220805.0,100108176,Swazi-Trac Pty Ltd,136763,,0.0,0.0,0.0,10177.94,,,,10177.94,67852.90
1,20221019.0,100108176,Swazi-Trac Pty Ltd,139441,,0.0,0.0,0.0,678.96,,,,678.96,4526.42
2,20230203.0,100240818,Serec Radio Ltd,25025,,0.0,0.0,0.0,352.41,,,,352.41,2349.40
3,20230301.0,100232150,Augusta Enterprises Swaziland,IN128486,,0.0,0.0,0.0,9135.73,,,,9135.73,60904.84
4,20230324.0,100232150,Augusta Enterprises Swaziland,IN128678,,0.0,0.0,0.0,21196.35,,,,21196.35,141309.00
5,20230417.0,100232150,Augusta Enterprises Swaziland,IN131458,,0.0,0.0,0.0,45263.77,,,,45263.77,301758.48
6,20230420.0,100199376,DHL International-Swaziland,MTSIR00026083,,0.0,0.0,0.0,0.00,,,,0.00,12558.82
7,20230420.0,100199376,DHL International-Swaziland,D0007562,,0.0,0.0,0.0,0.00,,,,0.00,-12558.82
8,20230510.0,100182626,Quality Investments t/a Quality Eng,7212,,0.0,0.0,0.0,1125.00,,,,1125.00,7500.00
9,20230510.0,100182626,Quality Investments t/a Quality Eng,7208,,0.0,0.0,0.0,825.00,,,,825.00,5500.00


In [29]:
with pd.ExcelWriter('C:\\Users\\bedlamini\\Documents\\VAT Schedules\\RES Files\\New VAT Schedule___Updated.xlsx',engine='openpyxl',mode='a',if_sheet_exists='overlay') as writer:
    df.to_excel(writer,sheet_name='VAT schedule form--input tab',startrow=2,header=False,index=False)